<a href="https://colab.research.google.com/github/rasecfaria/FinalProject/blob/main/v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Recommendation System 🎬

### Step 1: Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re  # Adicionando para extração do ano dos títulos

### Step 2: Load data

In [ ]:
ratings = pd.read_csv('data/ratings.csv')
movies = pd.read_csv('data/movies.csv')

# Função para extrair o ano do título do filme
def extract_year(title):
    # Procurar pelo ano entre parênteses no final do título
    match = re.search(r'\((\d{4})\)$', title)
    if match:
        return int(match.group(1))
    return None

# Adicionar coluna de ano ao DataFrame de filmes
movies['year'] = movies['title'].apply(extract_year)

# Calcular a distribuição de anos para entender a faixa de anos nos dados
year_counts = movies['year'].value_counts().sort_index()
min_year = movies['year'].min()
max_year = movies['year'].max()

print(f"Anos dos filmes: de {min_year} até {max_year}")
print(f"Total de filmes com ano identificado: {movies['year'].count()}")
print(f"Filmes sem ano identificado: {movies['year'].isna().sum()}")

### Step 3: Data analysis

In [ ]:
n_ratings = len(ratings)
n_movies = ratings['movieId'].nunique()
n_users = ratings['userId'].nunique()

print(f"Number of ratings: {n_ratings}")
print(f"Number of unique movies: {n_movies}")
print(f"Number of unique users: {n_users}")
print(f"Average number of ratings per user: {round(n_ratings/n_users, 2)}")
print(f"Average number of ratings per movie: {round(n_ratings/n_movies, 2)}")

### Step 3b: Normalização Centrada na Média

A normalização centrada na média consiste em ajustar as avaliações de cada usuário subtraindo a média de suas avaliações. Isso ajuda a corrigir o viés de alguns usuários que tendem a dar avaliações consistentemente mais altas ou mais baixas que outros.

A normalização é complementar à média bayesiana. Enquanto a média bayesiana ajuda a classificar filmes com poucas avaliações, a normalização ajuda a alinhar as escalas de avaliação entre diferentes usuários.

In [ ]:
def normalize_ratings(df):
    """
    Normaliza as avaliações subtraindo a média de avaliação de cada usuário.
    
    Args:
        df: DataFrame pandas contendo pelo menos 3 colunas (userId, movieId, rating)
        
    Returns:
        DataFrame normalizado contendo as colunas originais e uma coluna adicional 'rating_normalized'
    """
    # Calcula a média de avaliação de cada usuário
    user_mean_ratings = df.groupby('userId')['rating'].mean()
    
    # Cria uma cópia do DataFrame original
    df_normalized = df.copy()
    
    # Adiciona a coluna com as avaliações normalizadas
    df_normalized['rating_normalized'] = df_normalized.apply(
        lambda x: x['rating'] - user_mean_ratings[x['userId']], 
        axis=1
    )
    
    return df_normalized

# Aplicar normalização nas avaliações
ratings_normalized = normalize_ratings(ratings)

# Mostrar as primeiras linhas para verificar
ratings_normalized.head()

In [ ]:
from scipy.sparse import csr_matrix

def create_X(df, rating_column='rating'):
    """
    Generates a sparse matrix from ratings dataframe.

    Args:
        df: pandas dataframe containing at least 3 columns (userId, movieId, rating)
        rating_column: the name of the column to use for ratings (default: 'rating')

    Returns:
        X: sparse matrix
        user_mapper: dict that maps user id's to user indices
        user_inv_mapper: dict that maps user indices to user id's
        movie_mapper: dict that maps movie id's to movie indices
        movie_inv_mapper: dict that maps movie indices to movie id's
    """
    M = df['userId'].nunique()
    N = df['movieId'].nunique()

    user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(N))))

    user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(N)), np.unique(df["movieId"])))

    user_index = [user_mapper[i] for i in df['userId']]
    item_index = [movie_mapper[i] for i in df['movieId']]

    X = csr_matrix((df[rating_column], (user_index,item_index)), shape=(M,N))

    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

In [ ]:
# Criando a matriz com avaliações normalizadas
X_norm, user_mapper_norm, movie_mapper_norm, user_inv_mapper_norm, movie_inv_mapper_norm = create_X(ratings_normalized, rating_column='rating_normalized')

# Verificando a esparsidade da matriz normalizada
n_total_norm = X_norm.shape[0]*X_norm.shape[1]
n_ratings_norm = X_norm.nnz
sparsity_norm = n_ratings_norm/n_total_norm
print(f"Matrix sparsity (normalized): {round(sparsity_norm*100,2)}%")

### What is the distribution of movie ratings?

In [ ]:
sns.countplot(x='rating', data=ratings)
plt.title("Distribution of movie ratings", fontsize=14)
plt.xlabel('Rating') 
plt.ylabel('Number of movies')
plt.show()

In [ ]:
print(f"Mean global rating: {round(ratings['rating'].mean(),2)}.")

In [ ]:
mean_ratings = ratings.groupby('userId')[['rating']].mean()
print(f"Mean rating per user: {round(mean_ratings.mean(),2)}.")

### Which movies are most frequently rated?

In [ ]:
movie_ratings = ratings.merge(movies, on='movieId')
movie_ratings['title'].value_counts()[0:10]

### What are the lowest and highest rated movies?

Which movie has the lowest and highest average rating?

In [ ]:
mean_ratings = ratings.groupby('movieId')[['rating']].mean()
lowest_rated = mean_ratings['rating'].idxmin()

movies[movies['movieId'] == lowest_rated]

In [ ]:
highest_rated = mean_ratings['rating'].idxmax()
movies[movies['movieId'] == highest_rated]

#### Bayesian Average

[Bayesian Average](https://en.wikipedia.org/wiki/Bayesian_average) is defined as:

$r_{i} = \frac{C \times m + \Sigma{\text{reviews}}}{C+N}$

where $C$ represents our confidence, $m$ represents our prior, and $N$ is the total number of reviews for movie $i$.

- $C$ represents "the typical data set size". In this case, $C$ will be the average number of ratings for a given movie.
- $m$ represents the average rating across all movies

In [ ]:
movie_stats = ratings.groupby('movieId')['rating'].agg(['count', 'mean'])
movie_stats.head()

In [ ]:
C = movie_stats['count'].mean()
m = movie_stats['mean'].mean()

print(f"Average number of ratings for a given movie: {C:.2f}")
print(f"Average rating for a given movie: {m:.2f}")

def bayesian_avg(ratings):
    bayesian_avg = (C*m + ratings.sum())/(C+ratings.count())
    return round(bayesian_avg, 3)

In [ ]:
bayesian_avg_ratings = ratings.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']
movie_stats = movie_stats.merge(bayesian_avg_ratings, on='movieId')

In [ ]:
movie_stats.head()

In [ ]:
movie_stats = movie_stats.merge(movies[['movieId', 'title']])
movie_stats.sort_values(by='bayesian_avg', ascending=False)

In [ ]:
movie_stats.sort_values('bayesian_avg', ascending=True).head()

### A Glimpse at Movie Genres

The movies dataset needs to be cleaned in two ways:

- `genres` is expressed as a string with a pipe `|` separating each genre. We will manipulate this string into a list, which will make it much easier to analyze.
- `title` currently has (year) appended at the end. We will extract year from each title string and create a new column for it.

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))
movies.head()

**How many movie genres are there?**

In [ ]:
from collections import Counter

genre_frequency = Counter(g for genres in movies['genres'] for g in genres)

print(f"There are {len(genre_frequency)} genres.")

genre_frequency

#Gráfico da popularidade dos gêneros

In [ ]:
genre_frequency_df = pd.DataFrame([genre_frequency])
genre_frequency_df = genre_frequency_df.transpose()
genre_frequency_df = genre_frequency_df.reset_index()
genre_frequency_df.columns = ['genre', 'count']

sns.barplot(x='genre', y='count', data=genre_frequency_df.sort_values(by='count', ascending=False))
plt.xticks(rotation=90)
plt.title("Frequency of Movie Genres", fontsize=14)
plt.show()

### Step 4: Data Pre-processing

We are going to use a technique called colaborative filtering to generate recommendations for users. This technique is based on the premise that similar people like similar things.

The first step is to transform our data into a user-item matrix, also known as a "utility" matrix. In this matrix, rows represent users and columns represent movies. The beauty of collaborative filtering is that it doesn't require any information about the users or the movies user to generate recommendations.

<img src="https://github.com/rasecfaria/FinalProject/blob/main/images/user_movie_matrix.png?raw=1" width=50%/>

The `create_X()` function outputs a sparse matrix $X$ with four mapper dictionaries:

- **user_mapper**: maps user id to user index
- **movie_mapper**: maps movie id to movie index
- **user_inv_mapper**: maps user index to user id
- **movie_inv_mapper**: maps movie index to movie id

We need these dictionaries because they map which row/column of the utility matrix corresponds to which user/movie id.

Our $X$ (user-item) matrix is a [scipy.sparse.csr_matrix](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html) which stores the data sparsely.

In [ ]:
from scipy.sparse import csr_matrix

def create_X(df, rating_column='rating'):
    """
    Generates a sparse matrix from ratings dataframe.

    Args:
        df: pandas dataframe containing at least 3 columns (userId, movieId, rating)
        rating_column: the name of the column to use for ratings (default: 'rating')

    Returns:
        X: sparse matrix
        user_mapper: dict that maps user id's to user indices
        user_inv_mapper: dict that maps user indices to user id's
        movie_mapper: dict that maps movie id's to movie indices
        movie_inv_mapper: dict that maps movie indices to movie id's
    """
    M = df['userId'].nunique()
    N = df['movieId'].nunique()

    user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(N))))

    user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(N)), np.unique(df["movieId"])))

    user_index = [user_mapper[i] for i in df['userId']]
    item_index = [movie_mapper[i] for i in df['movieId']]

    X = csr_matrix((df[rating_column], (user_index,item_index)), shape=(M,N))

    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

# Criando a matriz com avaliações originais
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_X(ratings)

### Evaluating sparsity

Here, we calculate sparsity by dividing the number of stored elements by total number of elements. The number of stored (non-empty) elements in our matrix ([nnz](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.nnz.html)) is equivalent to the number of ratings in our dataset.

In [ ]:
n_total = X.shape[0]*X.shape[1]
n_ratings = X.nnz
sparsity = n_ratings/n_total
print(f"Matrix sparsity: {round(sparsity*100,2)}%")

`csr_matrix.nnz` counts the stored values in our sparse matrix. The rest of our cells are empty.

The **cold start problem** is when there are new users and movies in our matrix that do not have any ratings. In our Movielens dataset, all users and movies have at least one rating but in general, it's useful to check which users and movies have few interactions.

In [ ]:
n_ratings_per_user = X.getnnz(axis=1)
len(n_ratings_per_user)

In [ ]:
print(f"Most active user rated {n_ratings_per_user.max()} movies.")
print(f"Least active user rated {n_ratings_per_user.min()} movies.")

In [ ]:
n_ratings_per_movie = X.getnnz(axis=0)
len(n_ratings_per_movie)

In [ ]:
print(f"Most rated movie has {n_ratings_per_movie.max()} ratings.")
print(f"Least rated movie has {n_ratings_per_movie.min()} ratings.")

In [ ]:
plt.figure(figsize=(16,4))
plt.subplot(1,2,1)
sns.kdeplot(n_ratings_per_user, shade=True)
plt.xlim(0)
plt.title("Number of Ratings Per User", fontsize=14)
plt.xlabel("number of ratings per user")
plt.ylabel("density")
plt.subplot(1,2,2)
sns.kdeplot(n_ratings_per_movie, shade=True)
plt.xlim(0)
plt.title("Number of Ratings Per Movie", fontsize=14)
plt.xlabel("number of ratings per movie")
plt.ylabel("density")
plt.show()

### Step 5: Item-item Recommendations with k-Nearest Neighbors

We are going to find the $k$ movies that have the most similar user engagement vectors for movie $i$.

In [ ]:
from sklearn.neighbors import NearestNeighbors

def find_similar_movies(movie_id, X, movie_mapper, movie_inv_mapper, k, metric='cosine'):
    """
    Finds k-nearest neighbours for a given movie id.

    Args:
        movie_id: id of the movie of interest
        X: user-item utility matrix
        k: number of similar movies to retrieve
        metric: distance metric for kNN calculations

    Output: returns list of k similar movie ID's
    """
    X = X.T
    neighbour_ids = []

    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    if isinstance(movie_vec, (np.ndarray)):
        movie_vec = movie_vec.reshape(1,-1)
    # use k+1 since kNN output includes the movieId of interest
    kNN = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric)
    kNN.fit(X)
    neighbour = kNN.kneighbors(movie_vec, return_distance=False)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

`find_similar_movies()` takes in a `movieId` and `X` matrix, and outputs a list of $k$ movies that are similar to the `movieId` of interest.

Let's see how it works in action. We will first create another mapper that maps movieId to title so that our results are interpretable.

`find_similar_movies()` returns a list of `movieId`'s that are most similar to your movie of interest. Let's convert these id's to titles so that we can interpret our results. To make things easier, we will create a dictionary that maps `movieId` to `title`.

In [ ]:
movie_titles = dict(zip(movies['movieId'], movies['title']))

movie_id = 356

similar_movies = find_similar_movies(movie_id, X, movie_mapper, movie_inv_mapper, metric='cosine', k=10)
movie_title = movie_titles[movie_id]

print(f"Because you watched {movie_title}:")
for i in similar_movies:
    print(movie_titles[i])

### Comparação de recomendações com e sem normalização

Vamos comparar as recomendações obtidas usando a matriz original e a matriz normalizada.

In [ ]:
movie_id = 1

# Recomendações com matriz original
similar_movies_original = find_similar_movies(movie_id, X, movie_mapper, movie_inv_mapper, metric='cosine', k=10)
movie_title = movie_titles[movie_id]

print(f"Porque você assistiu {movie_title} (usando ratings originais):")
for i in similar_movies_original:
    print(movie_titles[i])

print("\n" + "-"*50 + "\n")

# Recomendações com matriz normalizada
similar_movies_norm = find_similar_movies(movie_id, X_norm, movie_mapper_norm, movie_inv_mapper_norm, metric='cosine', k=10)

print(f"Porque você assistiu {movie_title} (usando ratings normalizados):")
for i in similar_movies_norm:
    print(movie_titles[i])

The results above show the 10 movies that are most similar to Toy Story. Most movies in this list are family movies from the 1990s, which seems pretty reasonable. Note that these recommendations are based solely on user-item ratings. Movie features such as genres are not used in this approach.

You can also play around with the kNN distance metric and see what results you would get if you use "manhattan" or "euclidean" instead of "cosine".

In [ ]:
movie_id = 1

similar_movies = find_similar_movies(movie_id, X, movie_mapper, movie_inv_mapper, metric='euclidean', k=10)
movie_title = movie_titles[movie_id]

print(f"Because you watched {movie_title}:")
for i in similar_movies:
    print(movie_titles[i])

In [ ]:
# Escolhendo um usuário para demonstrar o efeito da normalização
sample_user_id = 1

user_ratings = ratings[ratings['userId'] == sample_user_id]
user_normalized_ratings = ratings_normalized[ratings_normalized['userId'] == sample_user_id]

print(f"Avaliações do usuário {sample_user_id}:")
user_ratings_with_titles = user_ratings.merge(movies[['movieId', 'title']], on='movieId')
print(user_ratings_with_titles[['title', 'rating']].head())

print("\nMédia de avaliação do usuário:", user_ratings['rating'].mean())

print("\nAvaliações normalizadas do usuário:")
user_norm_ratings_with_titles = user_normalized_ratings.merge(movies[['movieId', 'title']], on='movieId')
print(user_norm_ratings_with_titles[['title', 'rating', 'rating_normalized']].head())

### Step 6: Handling the cold-start problem

Collaborative filtering relies solely on user-item interactions within the utility matrix. The issue with this approach is that brand new users or items with no iteractions get excluded from the recommendation system. This is called the **cold start problem**. Content-based filtering is a way to handle this problem by generating recommendations based on user and item features.

First, we need to convert the `genres` column into binary features. Each genre will have its own column in the dataframe, and will be populated with 0 or 1.

In [ ]:
n_movies = movies['movieId'].nunique()
print(f"There are {n_movies} unique movies in our movies dataset.")

In [ ]:
genres = set(g for G in movies['genres'] for g in G)

for g in genres:
    movies[g] = movies.genres.transform(lambda x: int(g in x))

# Create movie_genres DataFrame dropping non-genre columns
movie_genres = movies.drop(columns=['movieId', 'title','genres'])

# Ensure the year column is also dropped as it might contain NaN values
if 'year' in movie_genres.columns:
    movie_genres = movie_genres.drop(columns=['year'])

# Fill any remaining NaN values with 0 (assuming 0 means the movie doesn't have that genre)
movie_genres = movie_genres.fillna(0)

In [ ]:
movie_genres.head()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Check for any remaining NaN values
if movie_genres.isna().any().any():
    print("Warning: NaN values found in movie_genres DataFrame. Filling with zeros.")
    movie_genres = movie_genres.fillna(0)

# Ensure all values are numeric
movie_genres = movie_genres.astype(float)

# Now compute the cosine similarity
cosine_sim = cosine_similarity(movie_genres, movie_genres)
print(f"Dimensions of our genres cosine similarity matrix: {cosine_sim.shape}")

As expected, after passing the `movie_genres` dataframe into the cosine_similarity() function, we get a cosine similarity matrix of shape $(n_{\text{movies}}, n_{\text{movies}})$.

This matrix is populated with values between 0 and 1 which represent the degree of similarity between movies along the x and y axes.

### Creating a movie finder function
Let's say we want to get recommendations for movies that are similar to Jumanji. To get results from our recommender, we need to know the exact title of a movie in our dataset.

In our dataset, Jumanji is actually listed as 'Jumanji (1995)'. If we misspell Jumanji or forget to include its year of release, our recommender won't be able to identify which movie we're interested in.

To make our recommender more user-friendly, we can use a Python package called [fuzzywuzzy](https://pypi.org/project/fuzzywuzzy/) which will find the most similar title to a string that you pass in. Let's create a function called `movie_finder()` which take advantage of fuzzywuzzy's string matching algorithm to get the most similar title to a user-inputted string.

In [ ]:
#!pip install fuzzywuzzy[speedup]

from fuzzywuzzy import process

def movie_finder(title):
    all_titles = movies['title'].tolist()
    closest_match = process.extractOne(title, all_titles)
    return closest_match[0]

Let's test this out with our Jumanji example.

In [ ]:
title = movie_finder('forest gump')
title

To get relevant recommendations for `Jumanji`, we need to find its index in the cosine simialrity matrix. To identify which row we should be looking at, we can create a movie index mapper which maps a movie title to the index that it represents in our matrix.

Let's create a movie index dictionary called `movie_idx` where the keys are movie titles and values are movie indices:

In [ ]:
movie_idx = dict(zip(movies['title'], list(movies.index)))
idx = movie_idx[title]
print(f"Movie index for {title}: {idx}")

Using this handy `movie_idx` dictionary, we know that Jumanji is represented by index 1 in our matrix. Let's get the top 10 most similar movies to Jumanji.

In [ ]:
n_recommendations=10
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:(n_recommendations+1)]

In [ ]:
similar_movies = [i[0] for i in sim_scores]
similar_movies

`similar_movies` is an array of indices that represents Jumanji's top 10 recommendations. We can get the corresponding movie titles by either creating an inverse movie_idx mapper or using iloc on the title column of the movies dataframe.

In [ ]:
print(f"Because you watched {title}:")
movies['title'].iloc[similar_movies]

Cool! These recommendations seem pretty relevant and similar to Jumanji. The first 5 movies are family-friendly films from the 90s.

We can test our recommender further with other movie titles. For your convenience, let's package the steps into a single function which takes in the movie title of interest and number of recommendations.

In [ ]:
def get_content_based_recommendations(title_string, n_recommendations=10):
    title = movie_finder(title_string)
    idx = movie_idx[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:(n_recommendations+1)]
    similar_movies = [i[0] for i in sim_scores]
    print(f"Because you watched {title}:")
    print(movies['title'].iloc[similar_movies])

In [ ]:
get_content_based_recommendations('toy story', 5)

### Step 7: Dimensionality Reduction with Matrix Factorization (advanced)

Matrix factorization (MF) is a linear algebra technique that can help us discover latent features underlying the interactions between users and movies. These latent features give a more compact representation of user tastes and item descriptions. MF is particularly useful for very sparse data and can enhance the quality of recommendations. The algorithm works by factorizing the original user-item matrix into two factor matrices:

- user-factor matrix (n_users, k)
- item-factor matrix (k, n_items)

We are reducing the dimensions of our original matrix into "taste" dimensions. We cannot interpret what each latent feature $k$ represents. However, we could imagine that one latent feature may represent users who like romantic comedies from the 1990s, while another latent feature may represent movies which are independent foreign language films.

$$X_{mn}\approx P_{mk}\times Q_{nk}^T = \hat{X} $$
<img src="https://github.com/rasecfaria/FinalProject/blob/main/images/matrix_factorization.png?raw=1" width="50%"/>

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=20, n_iter=10)
Q = svd.fit_transform(X.T)
Q.shape

In [ ]:
movie_id = 1
similar_movies = find_similar_movies(movie_id, Q.T, movie_mapper, movie_inv_mapper, k=10)
movie_title = movie_titles[movie_id]

print(f"Because you watched {movie_title}:")
for i in similar_movies:
    print(movie_titles[i])

### Aplicando SVD nos dados normalizados

Vamos aplicar a redução de dimensionalidade SVD aos dados normalizados para ver se há melhoria nas recomendações.

In [ ]:
from sklearn.decomposition import TruncatedSVD

# SVD com dados originais
svd_original = TruncatedSVD(n_components=20, n_iter=10)
Q_original = svd_original.fit_transform(X.T)

# SVD com dados normalizados
svd_norm = TruncatedSVD(n_components=20, n_iter=10)
Q_norm = svd_norm.fit_transform(X_norm.T)

# Comparando as variâncias explicadas
print(f"Variância explicada (dados originais): {svd_original.explained_variance_ratio_.sum():.4f}")
print(f"Variância explicada (dados normalizados): {svd_norm.explained_variance_ratio_.sum():.4f}")

In [ ]:
movie_id = 1  # Toy Story

# Recomendações com SVD original
similar_movies_svd_original = find_similar_movies(movie_id, Q_original.T, movie_mapper, movie_inv_mapper, metric='cosine', k=10)
movie_title = movie_titles[movie_id]

print(f"Porque você assistiu {movie_title} (usando SVD com ratings originais):")
for i in similar_movies_svd_original:
    print(movie_titles[i])

print("\n" + "-"*50 + "\n")

# Recomendações com SVD normalizado
similar_movies_svd_norm = find_similar_movies(movie_id, Q_norm.T, movie_mapper_norm, movie_inv_mapper_norm, metric='cosine', k=10)

print(f"Porque você assistiu {movie_title} (usando SVD com ratings normalizados):")
for i in similar_movies_svd_norm:
    print(movie_titles[i])

The results above are the most similar movies to Toy Story using kNN on our “compressed” movie-factor matrix. We reduced the dimensions down to n_components=20. We can think of each component representing a latent feature such as movie genre.

## Conclusão: Normalização Centrada na Média

A normalização centrada na média é uma técnica eficaz para melhorar sistemas de recomendação porque:

1. **Remove o viés do usuário**: Alguns usuários tendem a dar notas consistentemente mais altas ou mais baixas. A normalização nivela esse viés.

2. **Complementa a média bayesiana**: Enquanto a média bayesiana ajuda a lidar com filmes que têm poucas avaliações, a normalização centrada na média ajuda a lidar com diferentes escalas de avaliação entre usuários.

3. **Melhora a qualidade das recomendações**: Ao remover os vieses sistemáticos nas avaliações, a similaridade entre itens se torna mais precisa, resultando em recomendações mais relevantes.

4. **Aumenta a variância explicada no SVD**: Como observado nos resultados acima, a normalização geralmente permite que o SVD capture mais variância com o mesmo número de componentes.

Em resumo, a normalização centrada na média é uma etapa de pré-processamento crucial em sistemas de recomendação baseados em filtragem colaborativa, contribuindo significativamente para a qualidade das recomendações.

## Previsão de Ratings com Normalização

Uma das vantagens da normalização centrada na média é melhorar a precisão na previsão de ratings. Vamos implementar uma função simples para prever ratings e comparar os resultados com e sem normalização.

In [ ]:
def predict_rating(user_id, movie_id, X, user_mapper, movie_mapper, k=10):
    """
    Prevê o rating que um usuário daria a um filme usando filtragem colaborativa baseada em usuários.
    
    Args:
        user_id: ID do usuário
        movie_id: ID do filme
        X: Matriz de utilidade
        user_mapper: Mapeamento de ID do usuário para índice
        movie_mapper: Mapeamento de ID do filme para índice
        k: Número de vizinhos a considerar
        
    Returns:
        Rating previsto
    """
    if user_id not in user_mapper or movie_id not in movie_mapper:
        return None
    
    user_idx = user_mapper[user_id]
    movie_idx = movie_mapper[movie_id]
    
    # Extrai o vetor de ratings do usuário
    user_ratings = X[user_idx].toarray().flatten()
    
    # Encontra usuários semelhantes
    # Transposta para obter similaridade entre usuários
    user_similarities = X.dot(X[user_idx].T).toarray().flatten()
    user_similarities[user_idx] = 0  # Remove o próprio usuário
    
    # Pega os k usuários mais similares
    most_similar_users = np.argsort(user_similarities)[-k:]
    
    # Ratings dos usuários similares para o filme
    similar_users_ratings = []
    similar_users_similarities = []
    
    for similar_user_idx in most_similar_users:
        rating = X[similar_user_idx, movie_idx]
        if rating != 0:  # Apenas considera ratings não-zero
            similar_users_ratings.append(rating)
            similar_users_similarities.append(user_similarities[similar_user_idx])
    
    # Se não houver ratings similares, retorna None
    if len(similar_users_ratings) == 0:
        return None
    
    # Calcula o rating previsto como média ponderada
    prediction = np.average(similar_users_ratings, weights=similar_users_similarities)
    return prediction

In [ ]:
# Vamos escolher alguns usuários e filmes para testar a previsão
test_cases = [
    (1, 1),    # Usuário 1, Toy Story
    (1, 100),  # Usuário 1, outro filme
    (100, 1),  # Usuário 100, Toy Story
    (100, 100) # Usuário 100, outro filme
]

print("Previsões com dados originais:")
for user_id, movie_id in test_cases:
    prediction = predict_rating(user_id, movie_id, X, user_mapper, movie_mapper)
    title = movie_titles.get(movie_id, f"Filme {movie_id}")
    if prediction is not None:
        print(f"Usuário {user_id}, Filme '{title}': Rating previsto = {prediction:.2f}")
    else:
        print(f"Usuário {user_id}, Filme '{title}': Sem previsão disponível")

print("\nPrevisões com dados normalizados:")
for user_id, movie_id in test_cases:
    prediction = predict_rating(user_id, movie_id, X_norm, user_mapper_norm, movie_mapper_norm)
    title = movie_titles.get(movie_id, f"Filme {movie_id}")
    
    # Para normalização, precisamos adicionar a média do usuário de volta
    if prediction is not None:
        user_mean = ratings[ratings['userId'] == user_id]['rating'].mean()
        adjusted_prediction = prediction + user_mean
        print(f"Usuário {user_id}, Filme '{title}': Rating previsto = {adjusted_prediction:.2f} (normalizado: {prediction:.2f} + média: {user_mean:.2f})")
    else:
        print(f"Usuário {user_id}, Filme '{title}': Sem previsão disponível")

In [ ]:
from sklearn.metrics import mean_squared_error
import math

def evaluate_predictions(X, X_norm, user_mapper, movie_mapper, user_mapper_norm, movie_mapper_norm, n_samples=100):
    """
    Avalia a precisão das previsões com e sem normalização.
    
    Args:
        X: Matriz de utilidade original
        X_norm: Matriz de utilidade normalizada
        n_samples: Número de amostras para avaliar
        
    Returns:
        RMSE para ambos os modelos
    """
    # Seleciona ratings aleatórios para testar
    test_ratings = ratings.sample(n_samples)
    
    # Armazena previsões e valores reais
    actual_ratings = []
    predictions_original = []
    predictions_normalized = []
    
    for _, row in test_ratings.iterrows():
        user_id = row['userId']
        movie_id = row['movieId']
        actual_rating = row['rating']
        
        # Previsão com dados originais
        pred_original = predict_rating(user_id, movie_id, X, user_mapper, movie_mapper)
        
        # Previsão com dados normalizados
        pred_normalized = predict_rating(user_id, movie_id, X_norm, user_mapper_norm, movie_mapper_norm)
        
        # Ajusta a previsão normalizada adicionando a média do usuário
        if pred_normalized is not None:
            user_mean = ratings[ratings['userId'] == user_id]['rating'].mean()
            pred_normalized += user_mean
        
        # Apenas considera previsões válidas
        if pred_original is not None and pred_normalized is not None:
            actual_ratings.append(actual_rating)
            predictions_original.append(pred_original)
            predictions_normalized.append(pred_normalized)
    
    # Calcula RMSE para ambos os modelos
    rmse_original = math.sqrt(mean_squared_error(actual_ratings, predictions_original))
    rmse_normalized = math.sqrt(mean_squared_error(actual_ratings, predictions_normalized))
    
    return rmse_original, rmse_normalized, len(actual_ratings)

# Avalia a precisão dos modelos
rmse_original, rmse_normalized, n_evaluated = evaluate_predictions(
    X, X_norm, user_mapper, movie_mapper, user_mapper_norm, movie_mapper_norm, n_samples=500
)

print(f"Avaliação em {n_evaluated} ratings:")
print(f"RMSE (modelo original): {rmse_original:.4f}")
print(f"RMSE (modelo normalizado): {rmse_normalized:.4f}")
print(f"Melhoria com normalização: {(rmse_original - rmse_normalized) / rmse_original * 100:.2f}%")

## Observações Finais sobre a Normalização

A normalização centrada na média mostrou-se uma técnica eficaz para melhorar a qualidade das recomendações no nosso sistema de filmes. Observamos:

1. **Redução do RMSE**: A normalização ajudou a reduzir o erro quadrático médio nas previsões de ratings.

2. **Recomendações mais personalizadas**: Ao remover o viés individual de cada usuário, conseguimos capturar melhor suas preferências reais por diferentes tipos de filmes.

3. **Melhor desempenho do SVD**: A normalização permitiu que o SVD capturasse mais variância explicada com o mesmo número de componentes.

4. **Complementaridade com a média bayesiana**: A normalização centrada na média resolve um problema diferente da média bayesiana:
   - A **normalização** resolve o problema de usuários com diferentes escalas de avaliação
   - A **média bayesiana** resolve o problema de filmes com poucas avaliações

Para sistemas de recomendação em produção, é recomendável implementar ambas as técnicas para obter os melhores resultados.

### Outras técnicas de normalização a considerar:

Além da normalização centrada na média, existem outras técnicas que poderiam ser exploradas:

- **Normalização Z-Score**: (rating - média_usuário) / desvio_padrão_usuário
- **Normalização Min-Max**: (rating - min_usuário) / (max_usuário - min_usuário)
- **Normalização por frequência decrescente**: Pesos maiores para filmes menos vistos

Cada técnica tem suas vantagens e pode ser aplicada dependendo das características dos dados e dos objetivos do sistema de recomendação.

## Combinando Normalização Centrada na Média com Média Bayesiana

Podemos obter um sistema de recomendação ainda mais robusto combinando as duas técnicas:
1. **Normalização centrada na média**: Para corrigir o viés dos usuários
2. **Média bayesiana**: Para lidar com filmes que têm poucas avaliações

Vamos implementar uma função que utiliza ambas as técnicas para gerar recomendações.

In [ ]:
def get_recommendations_for_user(user_id, X_norm, user_mapper_norm, movie_mapper_norm, movie_inv_mapper_norm, 
                                 bayesian_avg_ratings=None, n_recommendations=10, min_year=1980):
    """
    Gera recomendações para um usuário específico usando normalização e média bayesiana.
    Filtra filmes muito antigos com base no parâmetro min_year.
    
    Args:
        user_id: ID do usuário
        X_norm: Matriz de utilidade normalizada
        user_mapper_norm: Mapeamento de ID do usuário para índice
        movie_mapper_norm: Mapeamento de ID do filme para índice
        movie_inv_mapper_norm: Mapeamento inverso de índice para ID do filme
        bayesian_avg_ratings: DataFrame com médias bayesianas dos filmes
        n_recommendations: Número de recomendações a gerar
        min_year: Ano mínimo para considerar filmes (para evitar filmes muito antigos)
        
    Returns:
        Lista de IDs de filmes recomendados
    """
    # Se bayesian_avg_ratings não for fornecido, recalcule-o
    if bayesian_avg_ratings is None:
        # Definir a função bayesian_avg localmente se não foi definida globalmente
        def bayesian_avg(ratings):
            C = ratings.count().mean()  # Média do número de avaliações
            m = ratings.mean().mean()   # Média das avaliações
            bayesian_avg = (C*m + ratings.sum())/(C+ratings.count())
            return round(bayesian_avg, 3)
        
        # Calcular as médias bayesianas para todos os filmes
        bayesian_avg_ratings = ratings.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
        bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']
    
    if user_id not in user_mapper_norm:
        return []
    
    user_idx = user_mapper_norm[user_id]
    user_ratings = X_norm[user_idx].toarray().flatten()
    
    # Filmes que o usuário já avaliou
    rated_movies_idx = np.where(user_ratings != 0)[0]
    rated_movies_ids = [movie_inv_mapper_norm[idx] for idx in rated_movies_idx]
    
    # Prevê ratings para todos os filmes não avaliados
    all_movie_indices = np.arange(X_norm.shape[1])
    unrated_movie_indices = np.setdiff1d(all_movie_indices, rated_movies_idx)
    
    # Calcular similaridade do usuário com outros usuários
    user_similarities = X_norm.dot(X_norm[user_idx].T).toarray().flatten()
    user_similarities[user_idx] = 0  # Remove o próprio usuário
    
    # Top k usuários mais similares
    k = 50  # Número de usuários similares a considerar
    most_similar_users = np.argsort(user_similarities)[-k:]
    
    # Para cada filme não avaliado, prevê o rating
    predictions = {}
    
    for movie_idx in unrated_movie_indices:
        movie_id = movie_inv_mapper_norm[movie_idx]
        
        # Verificar o ano do filme e pular filmes muito antigos
        movie_info = movies[movies['movieId'] == movie_id]
        if not movie_info.empty:
            movie_year = movie_info['year'].iloc[0]
            # Pular filmes sem ano ou com ano anterior ao mínimo especificado
            if pd.isna(movie_year) or movie_year < min_year:
                continue
        
        # Ratings dos usuários similares para o filme
        similar_users_ratings = []
        similar_users_similarities = []
        
        for similar_user_idx in most_similar_users:
            rating = X_norm[similar_user_idx, movie_idx]
            if rating != 0:  # Apenas considera ratings não-zero
                similar_users_ratings.append(rating)
                similar_users_similarities.append(user_similarities[similar_user_idx])
        
        # Se não houver ratings similares, pula este filme
        if len(similar_users_ratings) == 0:
            continue
        
        # Previsão como média ponderada
        predicted_norm_rating = np.average(similar_users_ratings, weights=similar_users_similarities)
        
        # Convertendo de volta para a escala original
        user_mean = ratings[ratings['userId'] == user_id]['rating'].mean()
        predicted_rating = predicted_norm_rating + user_mean
        
        # Combina com a média bayesiana
        bayesian_rating = bayesian_avg_ratings[bayesian_avg_ratings['movieId'] == movie_id]['bayesian_avg'].values
        if len(bayesian_rating) > 0:
            # Dá peso igual à previsão colaborativa e à média bayesiana
            combined_rating = 0.5 * predicted_rating + 0.5 * bayesian_rating[0]
            predictions[movie_id] = combined_rating
    
    # Ordena filmes por rating previsto
    sorted_predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
    recommended_movies = [movie_id for movie_id, _ in sorted_predictions[:n_recommendations]]
    
    return recommended_movies

# Teste da função para um usuário específico
test_user_id = 611  # Vamos usar o usuário 611 para o teste

# Definir o ano mínimo para filtrar filmes antigos
MIN_YEAR = 1980  # Somente filmes a partir de 1980

# Obtém recomendações
recommended_movies = get_recommendations_for_user(
    test_user_id, X_norm, user_mapper_norm, movie_mapper_norm, 
    movie_inv_mapper_norm, bayesian_avg_ratings, n_recommendations=10, min_year=MIN_YEAR
)

# Mostra as recomendações
print(f"Recomendações para o usuário {test_user_id} (filmes a partir de {MIN_YEAR}):")
for movie_id in recommended_movies:
    movie_info = movies[movies['movieId'] == movie_id]
    if not movie_info.empty:
        title = movie_info['title'].iloc[0]
        year = movie_info['year'].iloc[0]
        bayesian_rating = bayesian_avg_ratings[bayesian_avg_ratings['movieId'] == movie_id]['bayesian_avg'].values[0]
        print(f"- {title} (Ano: {year}, Média bayesiana: {bayesian_rating:.2f})")
    else:
        title = f"Filme {movie_id}"
        print(f"- {title}")

# Comparação com filmes já assistidos
user_watched_movies = ratings[ratings['userId'] == test_user_id]
user_watched_movies = user_watched_movies.merge(movies[['movieId', 'title', 'year']], on='movieId')
user_watched_movies = user_watched_movies.sort_values(by='rating', ascending=False)

print(f"\nFilmes que o usuário {test_user_id} já assistiu (melhores avaliações):")
for _, row in user_watched_movies.head(5).iterrows():
    year_info = f" ({row['year']})" if not pd.isna(row['year']) else ""
    print(f"- {row['title']}{year_info} (Rating: {row['rating']})")

## Considerações Finais

Implementamos com sucesso um sistema de recomendação de filmes com filtragem por ano. As principais contribuições foram:

1. **Normalização das avaliações**: Removemos o viés dos usuários subtraindo suas médias individuais.

2. **Média bayesiana**: Implementamos uma técnica para lidar com filmes que têm poucas avaliações.

3. **Filtragem por ano de lançamento**: Adicionamos a capacidade de filtrar filmes muito antigos, permitindo recomendações mais relevantes para o gosto contemporâneo.

4. **Parâmetro MIN_YEAR configurável**: Permitimos ao usuário ajustar facilmente o ano mínimo dos filmes recomendados, com valor padrão de 1980.

Esta implementação resolve os seguintes problemas:
- Diferentes escalas de avaliação entre usuários (via normalização)
- Filmes com poucas avaliações (via média bayesiana)
- Recomendações de filmes muito antigos (via filtro de ano)

Resultando em um sistema de recomendação mais preciso, personalizado e relevante para as preferências modernas.

## Configuração de Preferências

Você pode ajustar as configurações abaixo para personalizar suas recomendações de filmes.

In [ ]:
# Configurações personalizáveis

# Ano mínimo para filmes recomendados
# Altere este valor para ajustar a faixa de anos dos filmes que você deseja receber como recomendação
MIN_YEAR = 1980  # Padrão: filmes lançados a partir de 1980

# Garantir que min_year e max_year estejam definidos
if 'min_year' not in globals() or 'max_year' not in globals():
    min_year = int(movies['year'].min())
    max_year = int(movies['year'].max())

min_year = int(min_year)
max_year = int(max_year)

print(f"Você receberá recomendações de filmes lançados a partir de {MIN_YEAR}")

# Mostrando quantos filmes estão disponíveis nesta faixa de anos
available_movies = movies[movies['year'] >= MIN_YEAR]
n_available_movies = len(available_movies)
percent_available = (n_available_movies / movies['year'].notna().sum()) * 100

print(f"Existem {n_available_movies} filmes disponíveis para recomendação ({percent_available:.1f}% do total)")
print(f"Anos dos filmes no dataset: de {min_year} até {max_year}")

In [ ]:
def test_recommendations_for_user(user_id, min_year=1980, n_recommendations=10):
    """
    Função para testar recomendações para um usuário específico com um ano mínimo personalizado.
    
    Args:
        user_id: ID do usuário
        min_year: Ano mínimo para considerar filmes (padrão: 1980)
        n_recommendations: Número de recomendações a retornar
    """
    print(f"Gerando recomendações para o usuário {user_id} (filmes a partir de {min_year})...")
    
    # Verificar se o usuário existe no conjunto de dados
    if user_id not in ratings['userId'].unique():
        print(f"Usuário {user_id} não encontrado no conjunto de dados!")
        return []
    
    # Contar quantos filmes o usuário já avaliou
    user_ratings_count = ratings[ratings['userId'] == user_id].shape[0]
    print(f"O usuário {user_id} avaliou {user_ratings_count} filmes.")
    
    # Gerar recomendações
    recommended_movies = get_recommendations_for_user(
        user_id, X_norm, user_mapper_norm, movie_mapper_norm, 
        movie_inv_mapper_norm, None, 
        n_recommendations=n_recommendations, min_year=min_year
    )
    
    # Calcular a média bayesiana apenas uma vez
    bayesian_avg_ratings = ratings.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
    bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']
    
    # Mostrar as recomendações
    if recommended_movies:
        print(f"\nRecomendações para o usuário {user_id} (filmes a partir de {min_year}):")
        for i, movie_id in enumerate(recommended_movies):
            movie_info = movies[movies['movieId'] == movie_id]
            if not movie_info.empty:
                title = movie_info['title'].iloc[0]
                year = movie_info['year'].iloc[0]
                genres = movie_info['genres'].iloc[0]
                bayesian_rating = bayesian_avg_ratings[bayesian_avg_ratings['movieId'] == movie_id]['bayesian_avg'].values[0]
                print(f"{i+1}. {title} ({year}) - Gêneros: {genres} - Média bayesiana: {bayesian_rating:.2f}")
    else:
        print(f"\nNão foram encontradas recomendações para o usuário {user_id} com o filtro de ano mínimo {min_year}.")
    
    # Mostrar filmes que o usuário já assistiu com melhor avaliação
    user_watched_movies = ratings[ratings['userId'] == user_id]
    user_watched_movies = user_watched_movies.merge(movies[['movieId', 'title', 'year']], on='movieId')
    user_watched_movies = user_watched_movies.sort_values(by='rating', ascending=False)
    
    print(f"\nFilmes mais bem avaliados pelo usuário {user_id}:")
    for i, (_, row) in enumerate(user_watched_movies.head(5).iterrows()):
        year_info = f" ({row['year']})" if not pd.isna(row['year']) else ""
        print(f"{i+1}. {row['title']}{year_info} - Avaliação: {row['rating']}")
    
    return recommended_movies

# Parâmetros personalizáveis para teste
test_user_id = 611  # ID do usuário para testar
min_year_for_test = 1980  # Ano mínimo para recomendações (filtra filmes mais antigos)
n_recommendations = 15  # Número de recomendações a mostrar

# Executar o teste
_ = test_recommendations_for_user(test_user_id, min_year_for_test, n_recommendations)

# Teste comparativo mostrando como seria se não filterássemos por ano
print("\n" + "="*50 + "\n")
print("COMPARATIVO: Recomendações sem filtro de ano (mostrando apenas 5 para comparação)")
_ = test_recommendations_for_user(test_user_id, min_year=min_year, n_recommendations=5)

## Como Usar o Sistema de Recomendação

Este notebook implementa um sistema de recomendação de filmes que filtra filmes muito antigos, com as seguintes características:

1. **Filtragem por ano**: Evita recomendações de filmes muito antigos (como os de 1942 e 1954), permitindo definir um ano mínimo (padrão: 1980)

2. **Técnicas avançadas**: Combina normalização centrada na média e média bayesiana para melhores recomendações

3. **Flexibilidade**: Permite testar diferentes usuários e diferentes valores para o ano mínimo

### Para usar o sistema:

#### Definir o ano mínimo para filtragem:

```python
MIN_YEAR = 1980  # Altere este valor para filtrar filmes lançados antes desse ano
```

#### Obter recomendações para um usuário:

```python
test_recommendations_for_user(user_id=611, min_year=1980, n_recommendations=15)
```

### Ajustes personalizados:

- Para incluir filmes mais antigos, diminua o valor de `min_year` (ex: 1950)
- Para ver apenas filmes mais recentes, aumente o valor de `min_year` (ex: 2000)
- Para testar outros usuários, mude o valor de `user_id`

### Benefícios:

1. Os usuários recebem recomendações mais relevantes de filmes contemporâneos
2. O sistema evita recomendar filmes muito antigos que podem não agradar ao público atual
3. A configuração `MIN_YEAR` é facilmente ajustável para atender diferentes preferências

In [ ]:
# Demonstração clara do efeito do parâmetro MIN_YEAR na filtragem de filmes antigos

print("DEMONSTRAÇÃO: Efeito do parâmetro MIN_YEAR na filtragem de filmes antigos")
print("=" * 70)

# Teste com diferentes valores de MIN_YEAR
test_user_id = 611
years_to_test = [1930, 1960, 1980, 2000]

for year in years_to_test:
    print(f"\n\nTeste com MIN_YEAR = {year}:")
    print("-" * 50)
    
    # Obtenha as 5 principais recomendações para cada valor de MIN_YEAR
    recommended_movies = get_recommendations_for_user(
        test_user_id, X_norm, user_mapper_norm, movie_mapper_norm, 
        movie_inv_mapper_norm, bayesian_avg_ratings, 
        n_recommendations=5, min_year=year
    )
    
    # Mostre as 5 principais recomendações
    print(f"Top 5 recomendações para o usuário {test_user_id} (filmes a partir de {year}):")
    for i, movie_id in enumerate(recommended_movies):
        movie_info = movies[movies['movieId'] == movie_id]
        if not movie_info.empty:
            title = movie_info['title'].iloc[0]
            movie_year = movie_info['year'].iloc[0]
            print(f"{i+1}. {title} (Ano: {movie_year})")

print("\n\nObserve como o parâmetro MIN_YEAR filtra com sucesso os filmes mais antigos,")
print("permitindo apenas recomendações de filmes lançados a partir do ano especificado.")
print("Isso resolve o problema de receber recomendações de filmes muito antigos (como de 1942 ou 1954).")
